In [1]:
# Import Dependencies
from splinter import Browser
from bs4 import BeautifulSoup as soup
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager
import time
from selenium.webdriver.chrome.options import Options

In [16]:
# Set the executable path and initialize Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [/Users/curtissmith/.wdm/drivers/chromedriver/mac64/93.0.4577.63/chromedriver] found in cache


In [129]:
# Visit the Beer Advocate Top 250 page
url = 'https://www.beeradvocate.com/beer/popular/'
browser.visit(url)

In [130]:
# Convert the browser html to a soup object and then quit the browser
html = browser.html
top_250_home = soup(html, 'html.parser')

In [131]:
# find all a tags
all_a = top_250_home.find_all('a')

# select only relevant link (those with "beer/profile" in link)
links = [x for x in all_a if '/beer/profile/' in str(x)]

# iterate and get links to beers - even indices are links to beers, odd are links to brewery
beer_a_tags = []
for index, item in enumerate(links):
    if index % 2 == 0:
        beer_a_tags.append(item)
        
# get href url for navigating to beer webpage
beer_href = [x.get('href') for x in beer_a_tags]

### Loop Through Links to Retrieve Beer Information

In [132]:
start_time = time.time()

# initialize list of dictionaries
beer_info = []
errors =[]
count = 0
num_skipped = 0

#iterate through links
for link in beer_href:
    
    count += 1
    if count % 50 == 0:
        length = (time.time() - start_time)
        print(f'Elapsed Time: {length} seconds / {length/60} minutes')
        print(count)
    
    # navigate to link
    browser.click_link_by_href(link)
    
    # Parse the beer html with soup
    html = browser.html
    beer_soup = soup(html, 'html.parser')
    
    try:
        
        reviews = []
        
        spans = beer_soup.find_all('span', class_='Tooltip')
        num_reviews = spans[10].text
        num_reviews = int(num_reviews.replace(",", ""))
        
        # Only have beers with > 100 reviews
        if num_reviews < 100:
            print(num_reviews)
            num_skipped += 1
            continue

        sorting_links = [f"{link}?sort=top",
                        f"{link}?view=beer&sort=top&start=25",
                        f"{link}?view=beer&sort=top&start=50",
                        f"{link}?view=beer&sort=top&start=100"]


        for item in sorting_links:

            browser.click_link_by_href(item)

            html = browser.html
            beer_soup = soup(html, 'html.parser')

            # Get beer and brewery
            header = beer_soup.find('div', class_='titleBar')
            brewery = header.find("span").text
            beer_name = header.text.replace(brewery, "")
            beer_name = beer_name.replace('\n', "")

            # get ABV and rating
            spans = beer_soup.find_all('span', class_='Tooltip')
            abv = spans[2].text
            rating = spans[6].text

            # get style
            a = beer_soup.find_all('a', class_='Tooltip')
            style = a[4].text

            # get reviews
            review_div = beer_soup.find_all('div', {"id": "rating_fullview_content_2"})
            for item in review_div:
                reviews.append(item.text.split("%")[1])
#                 reviews.append(item)

    

        # create dictionary and add to list
        beer_info.append({"beer_name": beer_name,
                          "beer_style": style,
                          "brewery": brewery,
                          "abv": abv,
                          "rating": rating,
                          "reviews": reviews})

        #print(len(reviews))

        browser.visit(url)

        # Parse the beer html with soup
        html = browser.html
        top_250 = soup(html, 'html.parser')
    
    except Exception as exception:
        print(exception)
        errors.append(beer_name)
        browser.visit(url)
        print(errors)
        
        # Parse the beer html with soup
        html = browser.html
        top_250 = soup(html, 'html.parser')
    
length = (time.time() - start_time)
print(f'Script ran in {length} seconds / {length/60} minutes')
print(f'Skipped {num_skipped} beers')
print(f'There were {errors} errors')

50
100
150
200
250
Script ran in 1782.4382178783417 seconds / 29.707303631305695 minutes
Skipped 0 beers


NameError: name 'erros' is not defined

In [ ]:
"/beer/profile/1199/11757/?view=beer&sort=top&start=25"

In [121]:
len(beer_info)

4

In [122]:
df = pd.DataFrame(beer_info)

In [123]:
df.head()

,beer_name,beer_style,brewery,abv,rating,reviews
0,Breakfast Stout,Stout - Oatmeal,Founders Brewing Company,ABV:,Avg:,"[[[<b><a class=""username"" href=""/community/mem..."
1,90 Minute IPA,IPA - Imperial,Dogfish Head Craft Brewery,ABV:,Avg:,"[[[<b><a class=""username"" href=""/community/mem..."
2,Two Hearted Ale,IPA - American,Bell's Brewery - Eccentric Café & General Store,ABV:,Avg:,"[[[<b><a class=""username"" href=""/community/mem..."
3,Pliny The Elder,IPA - Imperial,Russian River Brewing Company,ABV:,Avg:,"[[[<b><a class=""username"" href=""/community/mem..."


In [124]:
df.to_csv("test.csv", index=False)